## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-08-11-14-54-28 +0000,nypost,"Trump federalizing DC police, deploying Nation...",https://nypost.com/2025/08/11/us-news/trump-fe...
1,2025-08-11-14-53-00 +0000,wsj,Ford’s Answer to Cheap Chinese EVs Starts With...,https://www.wsj.com/business/autos/ford-compac...
2,2025-08-11-14-50-25 +0000,nyt,A Sidelined Zelensky Warns That Russia Will Tr...,https://www.nytimes.com/2025/08/11/world/europ...
3,2025-08-11-14-50-03 +0000,startribune,Trump says he's placing Washington police unde...,https://www.startribune.com/trump-is-promising...
4,2025-08-11-14-50-00 +0000,wsj,EU Clears $4.8 Billion Prosus-Just Eat Deal Wi...,https://www.wsj.com/world/europe/eu-clears-4-7...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2350/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
0,trump,39
71,china,13
25,he,12
622,putin,10
281,new,9


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
171,2025-08-10-20-57-00 +0000,wsj,Intel CEO Lip-Bu Tan is set to visit the White...,https://www.wsj.com/tech/intel-ceo-trump-white...,87
147,2025-08-11-01-00-00 +0000,wsj,News the jobs market was much weaker than prev...,https://www.wsj.com/economy/jobs-inflation-tru...,86
3,2025-08-11-14-50-03 +0000,startribune,Trump says he's placing Washington police unde...,https://www.startribune.com/trump-is-promising...,78
92,2025-08-11-10-00-00 +0000,nypost,NY POSTcast Daily Debrief: Trump-Putin summit ...,https://nypost.com/2025/08/11/us-news/ny-postc...,77
156,2025-08-10-23-18-32 +0000,nypost,Intel CEO Lip-Bu Tan to visit White House on M...,https://nypost.com/2025/08/10/business/intel-c...,74


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
171,87,2025-08-10-20-57-00 +0000,wsj,Intel CEO Lip-Bu Tan is set to visit the White...,https://www.wsj.com/tech/intel-ceo-trump-white...
208,63,2025-08-10-16-41-54 +0000,nyt,Dmitri Kozak Was a Key Putin Aide. He Lost Pow...,https://www.nytimes.com/2025/08/10/world/europ...
128,41,2025-08-11-04-49-28 +0000,nypost,US military preparing to surge National Guard ...,https://nypost.com/2025/08/11/us-news/us-milit...
35,40,2025-08-11-13-46-00 +0000,wsj,Colombian Presidential Hopeful Dies Two Months...,https://www.wsj.com/world/americas/colombian-p...
212,40,2025-08-10-16-15-23 +0000,nypost,Netanyahu lays out Gaza City incursion goals a...,https://nypost.com/2025/08/10/world-news/netan...
54,35,2025-08-11-12-50-42 +0000,bbc,UN condemns targeted Israeli attack that kille...,https://www.bbc.com/news/articles/cq688qz3rlro...
144,33,2025-08-11-01-26-00 +0000,nyt,U.S. Government to Take Cut of Nvidia and AMD ...,https://www.nytimes.com/2025/08/10/technology/...
85,30,2025-08-11-10-27-12 +0000,nyt,"‘Fighting Fire With Fire,’ Democrats on Defens...",https://www.nytimes.com/2025/08/10/us/politics...
38,29,2025-08-11-13-34-29 +0000,nypost,Nuclear winter would bring mass starvation and...,https://nypost.com/2025/08/11/science/nuclear-...
147,29,2025-08-11-01-00-00 +0000,wsj,News the jobs market was much weaker than prev...,https://www.wsj.com/economy/jobs-inflation-tru...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
